In [2]:
import pandas as pd
import numpy as np
import re
%load_ext watermark
%watermark -iv -v -d

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
pandas      0.20.1
numpy       1.12.1
re          2.2.1
2018-02-15 

CPython 3.6.1
IPython 5.3.0


## Prevention is the best treatment
The easiest way of dealing with terribly formatted, poorly filled in spreadsheets, is to provide our collaborators with a sample spreadsheet with some rows already filled in by us with some dummy information.

# Read in our Data

Pandas comes with many different parsers, making our life a lot easier - luckily one of them handles excel files. The data we are dealing with here are modified from an original spreadsheet from a clinician, handed to one of my professors.

In [3]:
excelfile = pd.ExcelFile('./terrible_spreadsheet.xlsx')

In [4]:
firstsheet = excelfile.sheet_names[0]
excelfile.sheet_names

['Plate 1',
 'Plate 2',
 'Plate 3',
 'Plate 4',
 'Plate 5',
 'Plate6',
 'Plate7',
 'Plate8',
 'Plate9',
 'Plate10',
 'Plate11']

In [5]:
ff = pd.read_excel(excelfile, sheetname=firstsheet, header=1)
ff.shape

(47, 54)

To make our life easier, we want to read all worksheets from the spreadsheet into a single DataFrame. To keep track which row came from which worksheet, we will additionally incorporate a column with the name `sheet` into each DataFrame.

In [6]:
df = pd.concat([pd.read_excel(excelfile, sheetname=sheet, header=1).assign(sheet=sheet)
                for sheet in excelfile.sheet_names])

In [7]:
pd.DataFrame.assign?

In [8]:
df.shape

(470, 55)

## Clean the column names

For convenience in our further analysis, we want column names to only consist of numbers, letters or the underscore character. That is because columns for which this is true can be accessed via the `.`, so to access `column1`, we would write `df.column1`. We will use a regular expression or short regex to do this. For more info you can look at the slides from [Al Sweigart's talk "Yes it's time to learn regular expressions"](http://bitly.com/yesregex) or watch the talk itself.

In [9]:
df.columns

Index(['ABA', 'Age', 'BSA', 'Betatoxin', 'Exoprotein ext', 'Gender',
       'Glom.extract', 'HLA', 'HLA -2', 'HLA-1', 'HSA', 'Hemolysin gamma A',
       'Hemolysin gamma B', 'Hemolysin gamma C', 'Hospital ', 'LDL',
       'LukAB(Lab)', 'LukAB(cc30)', 'LukD', 'LukE', 'LukF-PV', 'LukS-PV',
       'PC-12', 'PC16', 'PC4', 'PLY', 'PNAG', 'PSM 4variant', 'PSMalpha2',
       'PSMalpha3', 'Pn CWPS', 'Pn PS12', 'Pn PS23', 'Rabbit IgG',
       'S.Pyogenese arcA', 'SEB', 'SEB.1', 'SEG', 'SEI', 'SEM', 'SEN', 'SEO',
       'SEU', 'SP', 'Sample ID', 'SpA domD5-WT', 'SpA domD5FcNull',
       'Tetanus Toxoid', 'Tetanus Toxoid.1', 'cytoplasmic ext', 'hIgA', 'hIgG',
       'psmalpah4', 'sheet', 'surface protein ext'],
      dtype='object')

In [10]:
oldcols = df.columns

In [11]:
type(oldcols)

pandas.core.indexes.base.Index

# ^^^
## 'oldcols' still point back to the dataframe, even if later on the column names are changed (?)

In [12]:
colcleaningregex = re.compile(r'[^\w]')   # r -- raw string

In [13]:
colcleaningregex.sub?

In [14]:
newcols = [colcleaningregex.sub('_', col.strip()) for col in df.columns]

In [15]:
newcols

['ABA',
 'Age',
 'BSA',
 'Betatoxin',
 'Exoprotein_ext',
 'Gender',
 'Glom_extract',
 'HLA',
 'HLA__2',
 'HLA_1',
 'HSA',
 'Hemolysin_gamma_A',
 'Hemolysin_gamma_B',
 'Hemolysin_gamma_C',
 'Hospital',
 'LDL',
 'LukAB_Lab_',
 'LukAB_cc30_',
 'LukD',
 'LukE',
 'LukF_PV',
 'LukS_PV',
 'PC_12',
 'PC16',
 'PC4',
 'PLY',
 'PNAG',
 'PSM_4variant',
 'PSMalpha2',
 'PSMalpha3',
 'Pn_CWPS',
 'Pn_PS12',
 'Pn_PS23',
 'Rabbit_IgG',
 'S_Pyogenese_arcA',
 'SEB',
 'SEB_1',
 'SEG',
 'SEI',
 'SEM',
 'SEN',
 'SEO',
 'SEU',
 'SP',
 'Sample_ID',
 'SpA_domD5_WT',
 'SpA_domD5FcNull',
 'Tetanus_Toxoid',
 'Tetanus_Toxoid_1',
 'cytoplasmic_ext',
 'hIgA',
 'hIgG',
 'psmalpah4',
 'sheet',
 'surface_protein_ext']

In [16]:
print(len(oldcols)-len(oldcols.unique()),
     len(newcols)-len(np.unique(newcols)))   # check if unique. Notice unique() vs np.unique()

0 0


In [17]:
df.columns = newcols

## Get rid of empty columns & clean index

Sometimes people include columns in their DataFrame that are completely useless for analysis, as they are completely empty. If we expect more data in the forms of additional spreadsheets to require our processing in the future, we would leave them in but since that is not the case here, we can simply delete those.

In [18]:
df.notnull().any()

ABA                     True
Age                     True
BSA                     True
Betatoxin               True
Exoprotein_ext          True
Gender                  True
Glom_extract            True
HLA                     True
HLA__2                  True
HLA_1                   True
HSA                     True
Hemolysin_gamma_A       True
Hemolysin_gamma_B       True
Hemolysin_gamma_C       True
Hospital                True
LDL                     True
LukAB_Lab_              True
LukAB_cc30_             True
LukD                    True
LukE                    True
LukF_PV                 True
LukS_PV                 True
PC_12                   True
PC16                    True
PC4                     True
PLY                     True
PNAG                    True
PSM_4variant            True
PSMalpha2               True
PSMalpha3               True
Pn_CWPS                 True
Pn_PS12                 True
Pn_PS23                 True
Rabbit_IgG              True
S_Pyogenese_ar

In [19]:
print(pd.notnull(np.nan))
print(pd.notnull(None))

False
False


In [20]:
df = df.loc[:, df.notnull().any()]

## Fill *Hospital*, *Age* and *Gender* columns

The `Hospital`, `Age` and `Gender` columns are only filled in every couple of lines. We want to fill in the blanks. Since we already have everything loaded into one DataFrame, we have to use a groupby operation. Otherwise the last info in these columns from one sheet can transfer to the empty rows in the beginning of the next sheet, which we do not want. **In order to be able to use apply on a groupby object, we need to `reset_index` because our DataFrame contains duplicate indices which is prohibited in groupby-apply operations.**

In [21]:
df.head(50)

,ABA,Age,BSA,Betatoxin,Exoprotein_ext,Gender,Glom_extract,HLA,HLA__2,HLA_1,...,Sample_ID,SpA_domD5_WT,SpA_domD5FcNull,Tetanus_Toxoid,cytoplasmic_ext,hIgA,hIgG,psmalpah4,sheet,surface_protein_ext
0,2630.00,NaN,NaN,9189.00,9250.00,NaN,573.00,6889.00,13300.00,14489.00,...,Standard 10,4746.00,5728.00,14945.00,9298.00,11785.00,17353.00,2416.00,Plate 1,9959.00
1,3502.00,NaN,NaN,19049.00,9587.00,NaN,619.00,6107.00,16852.00,18307.00,...,Standard 100,13215.00,10611.00,18060.00,4796.00,6289.00,19976.00,1342.00,Plate 1,15298.00
2,2402.00,NaN,NaN,20730.00,6598.00,NaN,645.00,3952.00,12894.00,13750.00,...,Standard 1000,16623.00,19865.00,15435.00,2227.00,1445.00,18957.00,677.00,Plate 1,9635.00
3,876.00,NaN,NaN,7920.00,2002.00,NaN,216.00,1120.00,4648.00,4414.00,...,Standard 10000,24401.00,15968.00,4918.00,778.00,671.00,18501.00,262.00,Plate 1,2777.00
4,440.00,NaN,NaN,3100.00,592.00,NaN,92.00,310.00,1393.00,1223.00,...,Standard 100000,28358.00,8467.00,1334.00,254.00,451.00,17731.00,113.00,Plate 1,799.00
5,195.00,NaN,NaN,1144.00,214.00,NaN,58.00,103.00,467.00,414.00,...,Standard 1000000,31612.00,3912.00,447.00,97.00,331.00,17346.00,64.00,Plate 1,324.00
6,96.00,NaN,NaN,478.00,81.00,NaN,37.00,34.00,159.00,135.00,...,Standard 10000000,30015.00,2011.00,130.00,38.00,201.00,15807.00,33.00,Plate 1,108.00
7,3342.00,60.0,NaN,15826.00,7549.00,M,651.00,6520.00,15372.00,18669.00,...,23234 V1 100,15747.00,14953.00,10981.00,1764.00,1985.00,18444.00,913.00,Plate 1,12459.00
8,2727.00,NaN,NaN,6026.00,6217.00,NaN,533.00,3149.00,8986.00,9116.00,...,23234 V1 1000,20498.00,22194.00,4387.00,1006.00,992.00,18933.00,585.00,Plate 1,7440.00
9,723.00,NaN,NaN,1472.00,1871.00,NaN,143.00,578.00,1507.00,1458.00,...,23234 V1 10000,26979.00,15318.00,535.00,231.00,461.00,18499.00,152.00,Plate 1,1780.00


In [22]:
df.loc[:, ['Hospital', 'Age', 'Gender']].iloc[0:20]

,Hospital,Age,Gender
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,hospital1,60.0,M
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [23]:
df = df.reset_index(drop=True) #required in order for the groupby-apply to work
df.loc[:, ['Hospital', 'Age', 'Gender']] = df.groupby('sheet').apply(
    lambda x: x.loc[:, ['Hospital', 'Age', 'Gender']].fillna(method='ffill')
)

In [24]:
##                                                        ^^^^^^^
## Notice the fillna(method='ffill')    <- 'forward fill'

In [25]:
pd.DataFrame.fillna?

In [27]:
df.loc[:, ['Hospital', 'Age', 'Gender']].iloc[0:20]

,Hospital,Age,Gender
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,hospital1,60.0,M
8,hospital1,60.0,M
9,hospital1,60.0,M


In [28]:
for name, frame in df.groupby('sheet'):
    print(name, frame.shape)

Plate 1 (47, 54)
Plate 2 (47, 54)
Plate 3 (47, 54)
Plate 4 (47, 54)
Plate 5 (47, 54)
Plate10 (39, 54)
Plate11 (8, 54)
Plate6 (47, 54)
Plate7 (47, 54)
Plate8 (47, 54)
Plate9 (47, 54)


## Extract *PatientID*, *Visit* and *Dilution* from *Sample_ID*

Three pieces of information are stored in the `Sample_ID` column - the `PatientID`, the `Visit` and the `Dilution`. 
The `PatientID` is a five digit number, the `Visit` comes in between the `PatientID` and the `Dilution` and the `Dilution` is composed of 1s and 0s and is at the end of the `Sample_ID` string.
Each of these can be missing in a row. We want to use a regular expression in order to extract this info.

In [29]:
df.Sample_ID.str?

In [30]:
#df.Sample_ID.str.extract(r'.*\s(?P<Dilution>1[0]+)', expand=True)
df.Sample_ID.str.extract(r'\s(?P<Visit>[^\s]*)\s', expand=False).unique()

array([nan, 'V1', 'V2', 'V3', 'v1', '', 'GS2', 'GS1', 'JM', 'VANDER'], dtype=object)

In [31]:
results = df.Sample_ID.str.extract(
    r'(?P<PatientID>\d{5})?\s*(?P<Visit>[^\s]+\d)?\s+(?P<Dilution>1[0]+)?\s*$', 
                         expand=True)
results

,PatientID,Visit,Dilution
0,NaN,NaN,10
1,NaN,NaN,100
2,NaN,NaN,1000
3,NaN,NaN,10000
4,NaN,NaN,100000
5,NaN,NaN,1000000
6,NaN,NaN,10000000
7,23234,V1,100
8,23234,V1,1000
9,23234,V1,10000


In [32]:
results.Visit.unique()

array([nan, 'V1', 'V2', 'V3', 'v1', 'GS2', 'GS1'], dtype=object)

In [33]:
results.PatientID.unique()

array([nan, '23234', '28531', '28729', '33142', '35568', '62901', '52950',
       '57756', '48689', '62129', '62300', '62900', '17588', '15363',
       '59707', '64779', '67029', '77612', '78202', '83700', '84504',
       '99361', '92827', '93954', '94232', '99382', '11825', '99624',
       '99682', '27764', '44989', '27986', '37422', '46713', '59302',
       '15439', '14127', '10732'], dtype=object)

In [34]:
results.Dilution.unique()

array(['10', '100', '1000', '10000', '100000', '1000000', '10000000', nan], dtype=object)